# Imports

In [136]:
import folium
from folium import plugins
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from datetime import datetime, timedelta
from itertools import chain
import heapq
from tqdm.notebook import tqdm
random.seed(0)
from tqdm import tqdm

In [ ]:
!pip install mapbox
!pip install h3

In [ ]:
from mapbox import MapMatcher
import h3

# Read files

In [ ]:
data1 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_jan_1_10.parquet")
data2 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_jan_11_20.parquet")
data3 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_jan_21_31.parquet")
data4 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_feb_1_10.parquet")
data5 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_feb_11_20.parquet")
data6 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_feb_21_28.parquet")
data7 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_mar_1_10.parquet")
data8 = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_mar_11_20.parquet")
bin_data = pd.read_parquet("../input/waste-management-parquet/varanasi_swm_bin.parquet")
data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8],ignore_index=True)

## Dustbin Data

In [ ]:
bin_data.head()

## Live Vehicles Data

In [ ]:
data.head()

# Preprocess Data

## Check Null Values

In [ ]:
print("Does Live Data have any null values? ",data.isnull().sum().any())
print("Does Dustbin Data have any null values? ",bin_data.isnull().sum().any())

## Drop Duplicates

In [ ]:
print(data.shape)
data = data.drop_duplicates()
print(data.shape)
print(bin_data.shape)
bin_data = bin_data.drop_duplicates()
print(bin_data.shape)

## Read Latitudes and Longitudes

In [ ]:
data['latitude']=data['location.coordinates'].apply(lambda x:float(x.split(",")[1][:-1].strip()))
data['longitude']=data['location.coordinates'].apply(lambda x:float(x.split(",")[0][1:].strip()))

bin_data['latitude']=bin_data['location.coordinates'].apply(lambda x:float(x.split(",")[1][:-1].strip()))
bin_data['longitude']=bin_data['location.coordinates'].apply(lambda x:float(x.split(",")[0][1:].strip()))

## Drop Columns

In [ ]:
data.drop(["location.coordinates", "location.type","id"], axis=1, inplace=True)

# Since all bins have same capacity, colour category and fullness threshold.
bin_data.drop(["binCapacity", "id","binColor","binCategory", "binFullnessThreshold", "location.type", "location.coordinates"], axis=1, inplace=True)

## Format DateTime

In [ ]:
data["observationDateTime"]=data['observationDateTime'].apply(lambda x:datetime.strptime(str(x.tz_convert("Asia/Kolkata")),'%Y-%m-%d %H:%M:%S%z'))
data["observationDateTime"]=data['observationDateTime'].apply(lambda x:x.tz_localize(None))

## Sort Data

In [ ]:
data.sort_values(by="observationDateTime")

## Remove Outlier Coordinates from Live Data

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = fig.add_gridspec(ncols=4, wspace=1)
ax = gs.subplots(sharex=True)

ax[0].boxplot(data["latitude"])
ax[1].boxplot(data["longitude"])
ax[2].boxplot(data["speed"])
ax[3].boxplot(data["bearing"])

ax[0].set_ylabel('Latitudes')
ax[1].set_ylabel('Longitudes')
ax[2].set_ylabel('Speed')
ax[3].set_ylabel('Bearing')

plt.show()

Latitudes have outliers near 28 while longitudes have outliers near 77.

In [ ]:
data = data[data["latitude"]<=26]
data.reset_index(inplace=True)

## Check Outlier Coordinates in static Dustbin Data

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = fig.add_gridspec(ncols=2, wspace=1)
ax = gs.subplots(sharex=True)

ax[0].boxplot(bin_data["latitude"])
ax[1].boxplot(bin_data["longitude"])


ax[0].set_ylabel('Latitudes')
ax[1].set_ylabel('Longitudes')


plt.show()

Dustbin data does not seem to have any outliers.

## Remove Vehicles with a very few data points

In [ ]:
vehicle_points_count = data['license_plate'].value_counts()
vehicle_points_count, vehicle_points_count.min(), vehicle_points_count.max(), vehicle_points_count.mean(), vehicle_points_count.median()

In [ ]:
min_data_points_req = 10000
for license_plate in data.license_plate.unique():
    if vehicle_points_count[license_plate] < min_data_points_req:
        data = data.drop(data[data.license_plate == license_plate].index)

In [ ]:
vehicle_points_count = data['license_plate'].value_counts()
vehicle_points_count, vehicle_points_count.min(), vehicle_points_count.max(), vehicle_points_count.mean(), vehicle_points_count.median()

# Exploring the Data

## Dustbin Locations and Area covered by Vehicles 

In [ ]:
rows = bin_data[["latitude", "longitude"]].to_numpy()
device_ids = bin_data["binID"].to_numpy()
co_ordinates =[]
lats = []
longs = []
for point in rows:
    lat = point[0]
    long = point[1]
    lats.append(lat)
    longs.append(long)
    co_ordinates.append([lat, long])
    
lats = np.array(lats)
longs = np.array(longs)

map_view = folium.Map(
    location=[lats.mean(), longs.mean()],
    zoom_start=15
)

for device_id, co_ordinate in zip(device_ids, co_ordinates):
    folium.Marker(
        location=co_ordinate,
        popup=device_id,
        icon=folium.Icon(icon="trash")
    ).add_to(map_view)

map_view.add_child(plugins.HeatMap(data[['latitude','longitude']].values, radius=15))
map_view.fit_bounds(map_view.get_bounds())
map_view

Vehicles cover a lot of area other than the locations of dustbins. Which means either they are parked there at the end of the day or they go outside the city to dump the waste or they collect garbage from other locations like residential and corporate areas.

**According to the [official document](https://acrobat.adobe.com/link/track?uri=urn:aaid:scds:US:ea4e1c8c-3e00-315a-92f0-ae45efc754db), there are many waste management plants setup outside the city where the vehicles dispose off the waste. That is why we see a lot of movement near the outskirts of the city too. Also, the vehicles pick up the waste from the apartments.**

## Daily Trend of Readings

In [ ]:
daily_trend = data.groupby(data.observationDateTime.dt.floor('1D')).count()["license_plate"]
daily_vehicles = data.groupby(data.observationDateTime.dt.floor('1D'))["license_plate"].nunique()

fig = plt.figure(figsize=(25,6))
gs = fig.add_gridspec(1,2, wspace=0.1)
ax = gs.subplots(sharex=True)

daily_trend.plot(kind='line',ax=ax[0])
daily_vehicles.plot(kind='line',ax=ax[1])

ax[0].set_ylabel('Daily Readings')
ax[1].set_ylabel('Number of Vehicles per Day')

plt.show()

**Daily Readings Plot:**

Daily Readings chart shows dip in the readings on March 18th because there was a holiday on the occasion of Holi.

**Number of Vehicles per day:**

You can see on Sundays and Public Holidays, the the number of vehicles collecting garbage are very low (around 80) and on Holi, the number drops even lower (around 60).
Although there are some Sundays when the number of vehicles is higher which is reasonable because Legislative Assembly elections were conducted in February and March in Uttar Pradesh.

For example:
1. 16 January: On the occasion of [Makar Sankranti](https://www.amarujala.com/uttar-pradesh/varanasi/makar-sankranti-2022-ganga-snan-varanasi-devotees-will-come-in-kashi-parking-made-these-places-from-14-to-16-january-see-route-diverson-plan), devotees come to Varanasi for the 'Ganga Snan'. So, the waste management staff needs to work round the clock to keep the city clean.
2. 23 January: The team worked on Sunday because it was holiday on 26th.
3. 27 February: PM had a rally in Varanasi on March 4. So, the waste management team worked on Sunday and also the number of vehicles increased.
4. 6 March: There was voting on 7th March. So they had to work on Sunday as they were on leave on 7th.

## Number of Vehicles and Types

In [ ]:
license_plates=np.unique(data.license_plate)
print(f'Number of Garbage Vehicles: {len(license_plates)}')
vehicle_data=data[['vehicleType','license_plate']].drop_duplicates().reset_index(drop=True)
print(vehicle_data['vehicleType'].value_counts())
fig,ax = plt.subplots(figsize=(25,6))
sns.countplot(data=vehicle_data,x='vehicleType',ax=ax)
plt.xlabel('Vehicle Type')
plt.ylabel('Number of Vehicles')
plt.show()

Auto tippers are more than all other vehicles combined as they are the major garbage collectors. Being small in size, they can easily navigate across the city and collect garbage.

## Bins not covered by Vehicles on a particular random day

In [ ]:
dates = []
for i in  data["observationDateTime"]:
    dates.append(str(i.date()))
dates = set(dates)

random_date = random.choice(list(dates))
print("Date selcted: ",random_date)

daily_data=data[(data["observationDateTime"] >= random_date+" 00:00:00") & (data["observationDateTime"] <= random_date+" 23:59:59")]

bin_wards = set(bin_data["wardID"].values)
daily_wards = set(daily_data["wardID"].values)
wards_not_visited = bin_wards - daily_wards
print("Wards not visited on {}: {} ".format(random_date,wards_not_visited))

wards_not_visited_df = pd.DataFrame()
for ward in wards_not_visited:
    wards_not_visited_df = pd.concat([wards_not_visited_df, bin_data[bin_data["wardID"]==ward]], ignore_index = True)

In [ ]:
wards_not_visited_df.head()

In [ ]:
rows = wards_not_visited_df[["latitude", "longitude"]].to_numpy()
device_ids = wards_not_visited_df["binID"].to_numpy()
co_ordinates =[]
lats = []
longs = []
for point in rows:
    lat = point[0]
    long = point[1]
    lats.append(lat)
    longs.append(long)
    co_ordinates.append([lat, long])
    
lats = np.array(lats)
longs = np.array(longs)

map_view = folium.Map(
    location=[lats.mean(), longs.mean()],
    zoom_start=15
)

for device_id, co_ordinate in zip(device_ids, co_ordinates):
    folium.Marker(
        location=co_ordinate,
        popup=device_id,
        icon=folium.Icon(icon="trash")
    ).add_to(map_view)

map_view.add_child(plugins.HeatMap(daily_data[['latitude','longitude']].values, radius=15))
map_view.fit_bounds(map_view.get_bounds())
map_view

## Analyzing Trips

In [ ]:
license_plate = "UP65HT3953"
data_single_vehicle=data[(data['license_plate']==license_plate) & (data["observationDateTime"]>="2022-02-06 00:00") & (data["observationDateTime"]<="2022-02-07 00:00")]
duty=[]
coord=[]
onflag=True
for i, row in data_single_vehicle.iterrows():
        coord.append([row['latitude'],row['longitude']])
        duty.append(coord)
        coord=[[row['latitude'],row['longitude']]]

map3=folium.Map()
coord=data_single_vehicle[['latitude','longitude']].values

for path in duty:
    folium.vector_layers.PolyLine(path,
                                  tooltip=license_plate,color='red'
                                  ,weight=5).add_to(map3)
folium.Marker(location=list(coord[0]),tooltip="start-06-02").add_to(map3)
folium.Marker(location=list(coord[-1]),tooltip='end-06-02').add_to(map3)
map3.fit_bounds(map3.get_bounds())


license_plate = "UP-65-KT-0412"
data_single_vehicle=data[(data['license_plate']==license_plate) & (data["observationDateTime"]>="2022-02-06 00:00") & (data["observationDateTime"]<="2022-02-07 00:00")]
duty=[]
coord=[]
onflag=True
for i, row in data_single_vehicle.iterrows():
        coord.append([row['latitude'],row['longitude']])
        duty.append(coord)
        coord=[[row['latitude'],row['longitude']]]

coord=data_single_vehicle[['latitude','longitude']].values

for path in duty:
    folium.vector_layers.PolyLine(path,
                                  tooltip=license_plate,color='purple'
                                  ,weight=5).add_to(map3)
folium.Marker(location=list(coord[0]),tooltip="start-06-02").add_to(map3)
folium.Marker(location=list(coord[-1]),tooltip='end-06-02').add_to(map3)
map3.fit_bounds(map3.get_bounds())

license_plate = "UP65KT0865"
data_single_vehicle=data[(data['license_plate']==license_plate) & (data["observationDateTime"]>="2022-02-06 00:00") & (data["observationDateTime"]<="2022-02-07 00:00")]
duty=[]
coord=[]
onflag=True
for i, row in data_single_vehicle.iterrows():
        coord.append([row['latitude'],row['longitude']])
        duty.append(coord)
        coord=[[row['latitude'],row['longitude']]]

coord=data_single_vehicle[['latitude','longitude']].values

for path in duty:
    folium.vector_layers.PolyLine(path,
                                  tooltip=license_plate,color='pink'
                                  ,weight=5).add_to(map3)
folium.Marker(location=list(coord[0]),tooltip="start-06-02").add_to(map3)
folium.Marker(location=list(coord[-1]),tooltip='end-06-02').add_to(map3)
map3.fit_bounds(map3.get_bounds())

license_plate = "UP65JT9928"
data_single_vehicle=data[(data['license_plate']==license_plate) & (data["observationDateTime"]>="2022-02-06 00:00") & (data["observationDateTime"]<="2022-02-07 00:00")]
duty=[]
coord=[]
onflag=True
for i, row in data_single_vehicle.iterrows():
        coord.append([row['latitude'],row['longitude']])
        duty.append(coord)
        coord=[[row['latitude'],row['longitude']]]

coord=data_single_vehicle[['latitude','longitude']].values

for path in duty:
    folium.vector_layers.PolyLine(path,
                                  tooltip=license_plate,color='blue'
                                  ,weight=5).add_to(map3)
folium.Marker(location=list(coord[0]),tooltip="start-06-02").add_to(map3)
folium.Marker(location=list(coord[-1]),tooltip='end-06-02').add_to(map3)
map3.fit_bounds(map3.get_bounds())

rows = bin_data[["latitude", "longitude"]].to_numpy()
device_ids = bin_data["binID"].to_numpy()
co_ordinates =[]
lats = []
longs = []
for point in rows:
    lat = point[0]
    long = point[1]
    lats.append(lat)
    longs.append(long)
    co_ordinates.append([lat, long])
    
lats = np.array(lats)
longs = np.array(longs)


for device_id, co_ordinate in zip(device_ids, co_ordinates):
    folium.Marker(
        location=co_ordinate,
        popup=device_id,
        icon=folium.Icon(icon="trash")
    ).add_to(map3)

map3.fit_bounds(map3.get_bounds())
map3

**UP65HT3953** and **UP-65-KT-0412** are refuse compactor and dumper respectively which, as you can see, travel on the almost fixed path everyday i.e. around the dustbins and then to a place outside city. This is obvious as they, being  huge vehicles, are supposed to collect trash from large bins and take it to waste management plants.

The other two,**UP65JT9928** and  **UP65KT0865**, are auto tippers, which travel to narrow lanes and streets distance during whole day and also their path is not pre-defined as they have to collect waste from a lot other places and then dump it in those tagged dustbins.

# Define Frequently Used Variables and Functions

In [ ]:
service = MapMatcher(access_token="pk.eyJ1Ijoic2VqYWxndXB0YSIsImEiOiJjbDZ2c2t3cGwwNGpqM2pwOWNtdG5oYml2In0.1mxdkNftoAkE4L2REGPGKg")

In [ ]:
vehicles = list(data['vehicleType'].unique())
vehicles_and_license_plates = {}
for vehicle in vehicles:
    vehicles_and_license_plates[vehicle] = list(vehicle_data[vehicle_data['vehicleType'] == vehicle]['license_plate'])

In [ ]:
def flatten_2d(lst):
    return sum(lst, [])

In [ ]:
def get_frequency_of_points(lst):
    d = {}
    for l in lst:
        t = tuple(l)
        if t in d:
            d[t] += 1
        else:
            d[t] = 1
    return d

In [ ]:
def get_vehicle_data_on_date(data_single_vehicle, month, day):
    return data_single_vehicle[(data_single_vehicle["observationDateTime"].dt.month == month) & (data_single_vehicle["observationDateTime"].dt.day == day)]

In [ ]:
def get_vehicle_data_for_day_shift(data_single_vehicle, month, day, start_time, end_time):
    if start_time < end_time:
        return data_single_vehicle[((data_single_vehicle["observationDateTime"].dt.month == month) & (data_single_vehicle["observationDateTime"].dt.day == day) & (data_single_vehicle["observationDateTime"].dt.hour >= start_time) & (data_single_vehicle["observationDateTime"].dt.hour < end_time))]
    else:
        return data_single_vehicle[((data_single_vehicle["observationDateTime"].dt.month == month) & (data_single_vehicle["observationDateTime"].dt.day == day) & (data_single_vehicle["observationDateTime"].dt.hour >= start_time)) | ((data_single_vehicle["observationDateTime"].dt.day == day+1) & (data_single_vehicle["observationDateTime"].dt.month == month) & (data_single_vehicle["observationDateTime"].dt.hour < end_time))]        

In [ ]:
def get_hex_code_for_point(lat, lon, resolution):
    return h3.geo_to_h3(lat, lon, resolution)

In [ ]:
def remove_duplicate_points_from_route(route):
    route = [tuple(coord) for coord in route]
    return list(set(route))

# Static Routes Analysis

## Helper Functions

In [ ]:
def get_map_matching_results_from_df(data, service):
    result = []
    number_of_data_points = len(data)
    for i in range(number_of_data_points//100 + 1):
        line = {"type": "Feature", "properties": {"coordTimes": [str(i).replace(" ", "T")+"Z" for i in list(data["observationDateTime"])[i*100:min((i+1)*100, number_of_data_points)]]}, "geometry": {"type": "LineString", "coordinates": [list(j) for j in list(zip(data['longitude'], data['latitude']))[i*100:min((i+1)*100, number_of_data_points)]]}}
        response = service.match(line, profile='mapbox.driving')
        result.append(response.geojson())
    return result

In [ ]:
def get_map_matching_results_from_list_of_coords(coords_list, service):
    result = []
    number_of_data_points = len(coords_list)
    for i in range(number_of_data_points//100 + 1):
        line = {"type": "Feature", "geometry": {"type": "LineString", "coordinates": coords_list[i*100:min((i+1)*100, number_of_data_points)]}}
        response = service.match(line, profile='mapbox.driving')
        result.append(response.geojson())
    return result

In [ ]:
def is_map_matching_result_valid(map_matching_result):
    for result in map_matching_result:
        if result['code'] != "Ok":
            return False
    return True

In [ ]:
def get_map_matched_route_segments(map_matching_results):
    route_segments = []
    for result in map_matching_results:
        for j in range(len(result['features'])):
            route_segments.append((result['features'][j]['geometry']['coordinates'], result['features'][j]['properties']['distance'], result['features'][j]['properties']['duration']))
    return route_segments

In [ ]:
def get_map_matched_continuous_route(map_matched_route_segments, service):
    route = []
    distance = sum(list(zip(*map_matched_route_segments))[1])
    duration = sum(list(zip(*map_matched_route_segments))[2])
    for i in range(len(map_matched_route_segments)-1):
        route += map_matched_route_segments[i][0]
        line = {"type": "Feature", "geometry": {"type": "LineString", "coordinates": [map_matched_route_segments[i][0][-1], map_matched_route_segments[i+1][0][0]]}}
        response = service.match(line, profile='mapbox.driving')
        if response.geojson()['code'] != "Ok":
            return ([], 0, 0)
        route += response.geojson()['features'][0]['geometry']['coordinates']
        distance += response.geojson()['features'][0]['properties']['distance']
        duration += response.geojson()['features'][0]['properties']['duration']
    route += map_matched_route_segments[len(map_matched_route_segments)-1][0]
    return route, distance, duration

In [ ]:
def get_general_route(all_routes, threshold, service):
    all_routes = [remove_duplicate_points_from_route(route) for route in all_routes]
    all_points_single_vehicle = flatten_2d(all_routes)
    number_of_routes = len(all_routes)
    points_frequency = get_frequency_of_points(all_points_single_vehicle)
    routes_common_points = []
    for point in points_frequency:
        if points_frequency[point] > int(threshold * number_of_routes):
            routes_common_points.append(list(point))
    map_matching_results = get_map_matching_results_from_list_of_coords(routes_common_points, service)
    if is_map_matching_result_valid(map_matching_results):
        route_segments = get_map_matched_route_segments(map_matching_results)
        continuous_route = get_map_matched_continuous_route(route_segments, service)
        if len(continuous_route[0]) > 0:
            return continuous_route
        else:
            route_segments = list(zip(*route_segments))
            return (list(route_segments[0]), sum(route_segments[1]), sum(route_segments[2]))
    else:
        return routes_common_points

In [ ]:
def plot_on_map_1d(map_name, color_name, weight_val, route):
    duty=[]
    coord=[]
    for k in route:
        coord.append(list(reversed(k)))
        duty.append(coord)
        coord=[list(reversed(k))]
    for path in duty:
        folium.vector_layers.PolyLine(path, color=color_name, weight=weight_val).add_to(map_name)
    map_name.fit_bounds(map_name.get_bounds())

In [ ]:
def plot_on_map_2d(map_name, color_name, weight_val, complete_route):
    for route in complete_route:
        duty=[]
        coord=[]
        for k in route:
            coord.append(list(reversed(k)))
            duty.append(coord)
            coord=[list(reversed(k))]
        for path in duty:
            folium.vector_layers.PolyLine(path, color=color_name, weight=weight_val).add_to(map_name)
        map_name.fit_bounds(map_name.get_bounds())

In [ ]:
def plot_points_on_map(map_name, colour_name, weight_val, radius_val, points_list):
    for point in points_list:
        folium.CircleMarker(location=[point[1], point[0]], radius=radius_val, weight=weight_val, color=colour_name).add_to(map_name)
    map_name.fit_bounds(map_name.get_bounds())

## Analysing Routes of Auto Tippers

### Usual Working Hours for Auto Tippers

In [ ]:
auto_tipper = vehicles_and_license_plates['Auto Tipper'][0]
auto_tipper_data = data[data['license_plate'] == auto_tipper]

# get number of days for which data is available
no_of_days = 0
for i in range(1, 4):
    for j in range(1, 31):
        auto_tipper_single_day_data = get_vehicle_data_on_date(auto_tipper_data, i, j)
        if len(auto_tipper_single_day_data) > 0:
            no_of_days += 1
            
# for every hour of the day, for how many days is the data missing for that hour
missing_data_hours_to_no_of_days = {}
for i in range(24):
    missing_data_hours_to_no_of_days[i] = 0
for i in range(1, 4):
    for j in range(1, 31):
        auto_tipper_single_day_data = get_vehicle_data_on_date(auto_tipper_data, i, j)
        if len(auto_tipper_single_day_data) > 0:
            for k in range(24):
                if (len(auto_tipper_single_day_data[auto_tipper_single_day_data['observationDateTime'].dt.hour == k]) == 0):
                    missing_data_hours_to_no_of_days[k] += 1
                    
for i in missing_data_hours_to_no_of_days:
    if missing_data_hours_to_no_of_days[i] > 0.7 * no_of_days:
        print(i)

In [ ]:
missing_data_hours_to_no_of_auto_tippers = {}
for auto_tipper in vehicles_and_license_plates['Auto Tipper']:
    auto_tipper_data = data[data['license_plate'] == auto_tipper]

    # get number of days for which data is available
    no_of_days = 0
    for i in range(1, 4):
        for j in range(1, 31):
            auto_tipper_single_day_data = get_vehicle_data_on_date(auto_tipper_data, i, j)
            if len(auto_tipper_single_day_data) > 0:
                no_of_days += 1

    # for every hour of the day, for how many days is the data missing for that hour
    missing_data_hours_to_no_of_days = {}
    for i in range(24):
        missing_data_hours_to_no_of_days[i] = 0
    for i in range(1, 4):
        for j in range(1, 31):
            auto_tipper_single_day_data = get_vehicle_data_on_date(auto_tipper_data, i, j)
            if len(auto_tipper_single_day_data) > 0:
                for k in range(24):
                    if (len(auto_tipper_single_day_data[auto_tipper_single_day_data['observationDateTime'].dt.hour == k]) == 0):
                        missing_data_hours_to_no_of_days[k] += 1

    for i in missing_data_hours_to_no_of_days:
        if missing_data_hours_to_no_of_days[i] > 0.7 * no_of_days:
            if i in missing_data_hours_to_no_of_auto_tippers:
                missing_data_hours_to_no_of_auto_tippers[i] += 1
            else:
                missing_data_hours_to_no_of_auto_tippers[i] = 1
                
missing_data_hours_to_no_of_auto_tippers, len(vehicles_and_license_plates['Auto Tipper'])

In [ ]:
for h in missing_data_hours_to_no_of_auto_tippers:
    if missing_data_hours_to_no_of_auto_tippers[h] > 0.7 * len(vehicles_and_license_plates['Auto Tipper']):
        print(h)

The working time range can be assumed to be 6 a.m. to 5 p.m.

### Observing Daily Routes for an Auto Tipper

In [ ]:
colours_list = ['black', 'blue', 'cyan', 'red', 'magenta']
auto_tipper = vehicles_and_license_plates['Auto Tipper'][0]
auto_tipper_data = data[data['license_plate'] == auto_tipper]
for i in range(1, 6):
    auto_tipper_data_single_day = get_vehicle_data_for_day_shift(auto_tipper_data, 1, i, 6, 17)
    map_matching_results = get_map_matching_results_from_df(auto_tipper_data_single_day, service)
    if is_map_matching_result_valid(map_matching_results):
        route = get_map_matched_continuous_route(get_map_matched_route_segments(map_matching_results), service)
        map3 = folium.Map()
        plot_on_map_1d(map3, colours_list[i-1], 5, route[0])
        display(map3)
    else:
        print("Map Matching could not be performed for", i, "January 2022.")

### Obtain a General Route

In [ ]:
auto_tipper = vehicles_and_license_plates['Auto Tipper'][0]
auto_tipper_data = data[data['license_plate'] == auto_tipper]
all_routes = []
for i in range(1, 31):
    auto_tipper_data_single_day = get_vehicle_data_for_day_shift(auto_tipper_data, 1, i, 6, 17)
    map_matching_results = get_map_matching_results_from_df(auto_tipper_data_single_day, service)
    if is_map_matching_result_valid(map_matching_results):
        route = get_map_matched_continuous_route(get_map_matched_route_segments(map_matching_results), service)
        all_routes.append(route[0])
general_route_result = get_general_route(all_routes, 0.8, service)
if len(general_route_result) != 3:
    print("Map Matching could not be performed for the general route. Plotting the points.")
    map3 = folium.Map()
    plot_points_on_map(map3, 'blue', 5, 2, general_route_result)
    display(map3)
else:
    route = general_route_result[0]
    if isinstance(route[0][0], list):
        print("A continuous route could not be obtained. Plotting the segments.")
        map3 = folium.Map()
        plot_on_map_2d(map3, 'blue', 5, route)
        display(map3)
    else:
        print("A continuous general route is obtained.")
        map3 = folium.Map()
        plot_on_map_1d(map3, 'blue', 5, route)
        display(map3)

# Dumping Locations

In [ ]:
dumping_locations = ['893c16454c3ffff',
                     '893c16454c7ffff',
                     '893c16454a3ffff',
                     '893c1645663ffff',
                     '893c1645243ffff',
                     '893c164524fffff',
                     '893c1645663ffff',
                     '893c1645637ffff',
                     '893c164562bffff',
                     '893c1644637ffff',
                     '893c16450d3ffff',
                     '893c164543bffff',
                     '893c16456a7ffff',
                     '893c164548bffff',
                     '893c164509bffff',
                     '893c1647277ffff',
                     '893c1647267ffff',
                     '893c16454a7ffff',
                     '893c16456afffff',
                     '893c164546fffff']

In [ ]:
map3 = folium.Map(zoom_start=10, location=[25.265972, 82.965890])
for hexagon in dumping_locations:
    mp=h3.h3_set_to_multi_polygon([hexagon], geo_json=True)
    h3s={"type": "Feature",
    "properties":{},
    "geometry": {"type": "MultiPolygon",
    "coordinates": mp}}
    g=folium.GeoJson(h3s,style_function=lambda x: {'fillColor': 'white','color':'black'})
    g.add_to(map3)
map3

# Find schedules for the given routes

## Helper functions to find schedules

In [ ]:
def get_route_in_hex(general_route, resolution):
    hex_route = []
    for point in general_route[0]:
        new_hex = h3.geo_to_h3(point[1], point[0], resolution)
        if len(hex_route) == 0 or new_hex != hex_route[-1]:
            hex_route.append(new_hex)
    return hex_route

In [ ]:
def get_hex_code_for_point(lat, lon, resolution):
    return h3.geo_to_h3(lat, lon, resolution)

In [ ]:
def get_timestamp_diff(vehicle_data_on_date, start_idx, end_idx):
    start_time = vehicle_data_on_date.iloc[start_idx]["observationDateTime"]
    end_time = vehicle_data_on_date.iloc[end_idx]["observationDateTime"]
    return (end_time - start_time).total_seconds()

In [ ]:
# Function to map a hex area from the general route to a data point in the actual data
# Taking all mappings
def map_hex_pair_to_point_pair(hex_point_start, hex_point_end, vehicle_data_on_date, resolution):
    data_len = len(vehicle_data_on_date)
    all_mappings = []
    for index in range(data_len - 1):
        row = vehicle_data_on_date.iloc[index]
        hex_code = get_hex_code_for_point(float(row['latitude']), float(row['longitude']), resolution)
        if hex_code == hex_point_start:
            for end_idx in range(index + 1,  data_len):
                new_row = vehicle_data_on_date.iloc[end_idx]
                new_hex = get_hex_code_for_point(float(new_row['latitude']), float(new_row['longitude']), resolution)
                if new_hex == hex_point_end:
                    timestamp_diff = get_timestamp_diff(vehicle_data_on_date, index, end_idx)
                    if timestamp_diff <= 3600*3:
                        all_mappings.append([index, end_idx, timestamp_diff])
    return all_mappings

Avg hex side for resolution 9 is ~200m.

In [ ]:
def get_avg_timestamp_diff_on_day(hex_start_point, hex_end_point, vehicle_data_on_date, resolution):
    all_mappings = map_hex_pair_to_point_pair(hex_start_point, hex_end_point, vehicle_data_on_date, resolution)
    avg_diff = 0
    for mapping in all_mappings:
        avg_diff += mapping[2]
    if len(all_mappings) != 0:
        return avg_diff/len(all_mappings)
    return None

In [131]:
def get_avg_timestamp_over_data(hex_start_point, hex_end_point, data_single_vehicle, resolution, all_days):
    all_times = []
    for date in all_days:
        day_diff = get_avg_timestamp_diff_on_day(hex_start_point, hex_end_point, get_vehicle_data_on_date(data_single_vehicle, date[0], date[1]), resolution)
        if day_diff != None:
            all_times.append(day_diff)
    if len(all_times) != 0:
        return sum(all_times)/len(all_times)
    return None

In [ ]:
def get_discont(data_single_vehicle, discont_hours):
    max_discont = 0
    start_discont = 0
    total_rows = len(data_single_vehicle)
    for index in range(total_rows - 1):
        time_diff = (data_single_vehicle.iloc[index + 1]["observationDateTime"] - data_single_vehicle.iloc[index]["observationDateTime"]).total_seconds()
        if time_diff > max_discont and time_diff < 3600 * discont_hours: # Getting a max hour discontinuity
            max_discont = time_diff
            start_discont = index
    return start_discont, max_discont/3600

In [ ]:
def get_start_time_for_day(data_single_vehicle, month, day):
    vehicle_day_data = get_vehicle_data_for_day_shift(data_single_vehicle, month, day, 7, 7)
    if len(vehicle_day_data) > 0:
        return vehicle_day_data.iloc[0]["observationDateTime"].time()
    return None

In [ ]:
def get_travel_time(start_point, end_point, service):
    origin = {'type': 'Feature', 'geometry': {'type': 'Point','coordinates': [start_point[1], start_point[0]]}}
    destination = {'type': 'Feature', 'geometry': {'type': 'Point','coordinates': [end_point[1], end_point[0]]}}
    return service.directions([origin, destination], 'mapbox.driving').geojson()['features'][0]['properties']['duration']

In [ ]:
def get_avg_start_time(data_single_vehicle):
    sum_seconds = 0
    num_seconds = 0
    for date in all_days:
        start_time = get_start_time_for_day(data_single_vehicle, date[0], date[1])
        if start_time is not None:
            seconds = (start_time.hour * 3600) + (start_time.minute * 60) + (start_time.second)
            sum_seconds += seconds
            num_seconds += 1
    return pd.to_datetime(sum_seconds//num_seconds, unit='s')

In [ ]:
def plot_schedule(schedule, map3):
    for [hexagon, timestamp] in schedule:
        mp=h3.h3_set_to_multi_polygon([hexagon], geo_json=True)
        h3s={"type": "Feature",
        "properties":{},
        "geometry": {"type": "MultiPolygon",
        "coordinates": mp}}
        g=folium.GeoJson(h3s,style_function=lambda x: {'fillColor': 'white','color':'black'})
        folium.Marker(h3.h3_to_geo(hexagon), popup=str(timestamp.hour) + ":" + str(timestamp.minute) + ":" + str(timestamp.second)).add_to(map3)
        g.add_to(map3)

In [ ]:
def plot_day_schedule(data_single_vehicle, month, day, resolution, map3):
    day_data = get_vehicle_data_for_day_shift(data_single_vehicle, month, day)
    total_data_points = len(day_data)
    for i in range(total_data_points):
        curr_data = data_single_vehicle.iloc[i]
        hexagon = get_hex_code_for_point(curr_data['latitude'], curr_data['longitude'], resolution)
        mp=h3.h3_set_to_multi_polygon([hexagon], geo_json=True)
        h3s={"type": "Feature",
        "properties":{},
        "geometry": {"type": "MultiPolygon",
        "coordinates": mp}}
        g=folium.GeoJson(h3s,style_function=lambda x: {'fillColor': 'white','color':'black'})
        timestamp = curr_data["observationDateTime"]
        folium.Marker(h3.h3_to_geo(hexagon), popup=str(timestamp.hour) + ":" + str(timestamp.minute) + ":" + str(timestamp.second)).add_to(map3)
        g.add_to(map3)

In [ ]:
def get_seconds(time):
    return (time.hour * 3600) + (time.minute * 60) + time.second

In [ ]:
def find_closest_time_diff(schedule, curr_time):
    min_time_diff = abs(get_seconds(schedule[0][1]) - get_seconds(curr_time))
    min_time_diff_entry = schedule[0]
    for entry in schedule:
        time_diff = abs(get_seconds(entry[1]) - get_seconds(curr_time))
        if time_diff < min_time_diff:
            min_time_diff = time_diff
            min_time_diff_entry = entry
    return (min_time_diff_entry, min_time_diff)

In [142]:
def good_schedule_metric(data_single_vehicle, month, day, resolution, schedule):
    good_schedules = 0
    total_matching_schedules = 0
    day_data = get_vehicle_data_for_day_shift(data_single_vehicle, month, day, 7, 16)
    total_data_points = len(day_data)
    for i in range(total_data_points):
        curr_data = data_single_vehicle.iloc[i]
        curr_time = curr_data["observationDateTime"]
        closest_time_diff_entry = find_closest_time_diff(schedule, curr_time)
        if closest_time_diff_entry[1] < 20 * 3600:
            hex_code_for_point = get_hex_code_for_point(curr_data['latitude'], curr_data['longitude'], resolution)
            if closest_time_diff_entry[0][0] == hex_code_for_point or h3.h3_indexes_are_neighbors(closest_time_diff_entry[0][0], hex_code_for_point):
                good_schedules += 1
            total_matching_schedules += 1
    if total_matching_schedules == 0:
        return None
    return good_schedules/total_matching_schedules

In [ ]:
def get_all_days():
    all_days = []
    for i in range(1, 32):
        all_days.append([1, i])
    for i in range(1, 29):
        all_days.append([2, i])
    for i in range(1, 21):
        all_days.append([3, i])
    return all_days

## Find schedules

In [172]:
def get_route_from_list(data, service):
    complete_route = []
    number_of_data_points = len(data)
    for i in range(number_of_data_points//100 + 1):
        line = {"type": "Feature", "geometry": {"type": "LineString", "coordinates": data[i*100:min((i+1)*100, number_of_data_points)]}}
        response = service.match(line, profile='mapbox.driving')
        if response.geojson()['code'] == "Ok":
            complete_route.append(response.geojson()['features'][0]['geometry']['coordinates'])
    return complete_route

In [173]:
def get_route(data, service):
    complete_route = []
    number_of_data_points = len(data)
    for i in range(number_of_data_points//100 + 1):
        line = {"type": "Feature", "properties": {"coordTimes": [str(i).replace(" ", "T")+"Z" for i in list(data["observationDateTime"])[i*100:min((i+1)*100, number_of_data_points)]]}, "geometry": {"type": "LineString", "coordinates": [list(j) for j in list(zip(data['longitude'], data['latitude']))[i*100:min((i+1)*100, number_of_data_points)]]}}
        response = service.match(line, profile='mapbox.driving')
        if response.geojson()['code'] == "Ok":
            complete_route.append(response.geojson()['features'][0]['geometry']['coordinates'])
    return complete_route

In [174]:
def get_routes_for_all_days(data_single_vehicle):
    all_routes = []
    for j in range(1, 4):
        for i in range(1, 32):
            data_single_day = get_vehicle_data_on_date(data_single_vehicle, j, i)
            route_coords_day = sum(get_route(data_single_day, service), [])
            if len(route_coords_day) > 0:
                all_routes.append(route_coords_day)
    return all_routes

In [175]:
def get_general_route(data_single_vehicle, threshold):
    all_routes_single_vehicle = get_routes_for_all_days(data_single_vehicle)
    all_points_single_vehicle = flatten_2d(all_routes_single_vehicle)
    number_of_routes = len(all_routes_single_vehicle)
    points_frequency = get_frequency_of_points(all_points_single_vehicle)
    routes_common_points = []
    for point in points_frequency:
        if points_frequency[point] > int(threshold * number_of_routes):
            routes_common_points.append(list(point))
    common_route = get_route_from_list(routes_common_points, service)
    return common_route

In [176]:
resolution = 9
all_days = get_all_days()
vehicle = 'Hydraulic Lifter'
license_plate = vehicles_and_license_plates[vehicle][0]
data_single_vehicle = data[data['license_plate'] == license_plate]

In [185]:
general_route = flatten_2d(get_general_route(data_single_vehicle, 0.5))

In [187]:
len(general_route)

1446

The day shift mostly starts from somewhere between 7 am to 10am

In [188]:
hex_route = get_route_in_hex(general_route, resolution)

In [189]:
route_len = len(hex_route)
route_len

155

In [190]:
start_time = get_avg_start_time(data_single_vehicle)
start_time

Timestamp('1970-01-01 13:06:35')

In [191]:
import datetime

In [192]:
schedule = [(hex_route[0], start_time)]
last_point_taken = hex_route[0]
for i in tqdm(range(route_len - 1)):
    time_diff = get_avg_timestamp_over_data(last_point_taken, hex_route[i+1], data_single_vehicle, resolution, all_days)
    if time_diff != None:
        last_point_taken = hex_route[i+1]
        new_time = schedule[-1][1] + datetime.timedelta(seconds=time_diff)
        schedule.append((last_point_taken, new_time))

100%|██████████| 154/154 [16:19<00:00,  6.36s/it]


In [197]:
map3 = folium.Map(zoom_start=14, location=[25.265972, 82.965890])
for [hexagon, timestamp] in schedule:
    mp=h3.h3_set_to_multi_polygon([hexagon], geo_json=True)
    h3s={"type": "Feature",
    "properties":{},
    "geometry": {"type": "MultiPolygon",
    "coordinates": mp}}
    g=folium.GeoJson(h3s,style_function=lambda x: {'fillColor': 'white','color':'black'})
    folium.Marker(h3.h3_to_geo(hexagon), popup=str(timestamp.hour) + ":" + str(timestamp.minute) + ":" + str(timestamp.second)).add_to(map3)
    g.add_to(map3)
    
map3

In [198]:
all_metrics = []
for day in all_days:
    metric = good_schedule_metric(data_single_vehicle, day[0], day[1], resolution, schedule)
    if metric is not None:
        all_metrics.append(metric)

In [199]:
print("Min good metric", min(all_metrics))
print("Max good metric", max(all_metrics))
print("Mean metric", sum(all_metrics)/len(all_metrics))

Min good metric 0.0
Max good metric 0.06
Mean metric 0.023438904104372812
